In [1]:
# Imports
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from clean_up import data_cleaning

In [2]:
# Instantiate playlist DataFrame objects
playlist_1 = data_cleaning(r'C:\Users\Alex Lucchesi\OneDrive\Desktop\playlist_data\playlist1.csv')
playlist_2 = data_cleaning(r'C:\Users\Alex Lucchesi\OneDrive\Desktop\playlist_data\playlist2.csv')

In [4]:
# We can merge both dataframe objects to get all songs in one dataframe object.
merge = pd.merge(playlist_1, playlist_2, how = 'outer')

# Data Analysis and Visualization Questions:

## Runtime?

### What song has the longest runtime?
By using the duration column and the merged DataFrame object, we can identify the song which has the longest duration between both playlists.

In [5]:
# Grab the song with the largest duration of both playlists
merge[merge['duration'] == max(merge['duration'])]

,album,artists,duration,explicit,name,popularity,preview_url
31,Ocean,John Butler,11:39,False,Ocean,49,https://p.scdn.co/mp3-preview/182e1fc3028c04f5...


After running the code above, we can see the song Ocean by John Butler is the song with the longest duration. The duration for this song is 11 minutes and 39 seconds! 

### What is the average runtime of songs in each playlist?
### Which playlist has a longer total runtime?

Using both DataFrame objects, we can identify the average runtime of each playlist using the duration column. In the calculation, it will be easy to grab both answers at the same time as we will create a sum of total time in each duration column. 

In [6]:
# Create time variables for each DataFrame to store sum of time in seconds
time_1=0
time_2=0

# Iterate through both duration columns
for dur1, dur2 in zip(playlist_1['duration'], playlist_2['duration']):
    # Split into minutes and seconds
    m1, s1 = dur1.split(':')
    # Create total time and add it to playlist total
    total1 = 60 * int(m1) + int(s1)
    time_1 += total1

    # Split into minutes and seconds
    m2, s2 = dur2.split(':')
    # Create total time and add it to playlist total
    total2 = 60 * int(m2) + int(s2)
    time_2 += total2
# Calculate averages using both time sum variables
time_1_avg = time_1 / len(playlist_1['duration'])
time_2_avg = time_2 / len(playlist_2['duration'])

print(f"Average times for Playlist 1 are {round(time_1_avg,2)} seconds while average times for Playlist 2 are {round(time_2_avg, 2)} seconds")
print(f"Playlist 1 has a runtime of {time_1} seconds while Playlist 2 has a runtime of {time_2} seconds")
print(f"Playlist 1 has a slightly longer runtime. The difference between both is {time_1 - time_2} seconds")

Average times for Playlist 1 are 203.52 seconds while average times for Playlist 2 are 217.16 seconds
Playlist 1 has a runtime of 43553 seconds while Playlist 2 has a runtime of 43431 seconds
Playlist 1 has a slightly longer runtime. The difference between both is 122 seconds


### Can we visualize the song runtime?


In [7]:
# Create top 10 duration DataFrame from both playlist
top_10_duration = merge[merge['name'].duplicated() == False].sort_values(by = 'duration').tail(10)


In [8]:
# Visualize top 10
px.bar(top_10_duration, x = 'duration', y = 'name', hover_data= ['artists', 'explicit', 'popularity'], orientation='h')

In [9]:
# Visualize duration of all songs
px.histogram(merge.sort_values(by = 'duration'), x = 'duration', hover_data=['name', 'artists', 'album'])

## Popularity

### Which playlist contains more popular music?


We can view the playlists with duplicated data or with singular songs in it. 

We could argue that the playlist data from multiple users, therefore leaving in duplicates allows insight into the total popularity of a song. However, we could also argue that leaving in duplicate values detaches us from the real popularity of each playlist. For this assesment, I chose to leave in duplicates.

In [10]:
#Popularity is an integer column. We can sum up the values in each column and compare to one another
p1_pop = playlist_1['popularity'].sum()
p2_pop = playlist_2['popularity'].sum()

print(f'Playlist 1 contains a total popularity score of {p1_pop}\nPlaylist 2 contains a total popularity score of {p2_pop}')
print(f'Playlist 2 has a higher total popularity score, totaling {p2_pop}, which is higher than Playlist 1 by {p2_pop - p1_pop}')

Playlist 1 contains a total popularity score of 8788
Playlist 2 contains a total popularity score of 10878
Playlist 2 has a higher total popularity score, totaling 10878, which is higher than Playlist 1 by 2090


### Create a visualization of song popularity

In [11]:
px.histogram(merge, y='name', x='popularity', orientation='h', height = 1000, title = 'Sum of Popularity by Song').update_layout(title_x = 0.5)

## Duplicates

### Which playlist contains more duplicate songs?

In [12]:
# Apply boolean mask across DataFrames to get duplicated values
#Sum up the number of values that appear in the column to get number of duplicate songs
dupe_1 = playlist_1[playlist_1.duplicated() == True].value_counts().sum()
dupe_2 = playlist_2[playlist_2.duplicated() == True].value_counts().sum()
print(f'The number of duplicate songs in playlist 1 is {dupe_1}\nThe number of duplicate songs in playlist 2 is {dupe_2}')
print(f'The playlist with the most duplicated songs is playlist 2, totaling {dupe_2}, which is {dupe_2 - dupe_1} more than playlist 1')

The number of duplicate songs in playlist 1 is 22
The number of duplicate songs in playlist 2 is 45
The playlist with the most duplicated songs is playlist 2, totaling 45, which is 23 more than playlist 1


### Which song is present the most times across both playlists?

In [13]:
# Value counts can return the song with the highest frequency.
# Calling head and returning only one value on the series can give the top song
top_1 = playlist_1['name'].value_counts().head(1).rename_axis('Top Song').reset_index(name = 'counts')
top_2 = playlist_2['name'].value_counts().head(1).rename_axis('Top Song').reset_index(name = 'counts')
top_both = merge['name'].value_counts().head(1).rename_axis('Top Song').reset_index(name = 'counts')

print(f'The top song in playlist 1 is {top_1["Top Song"].values[0]}\nThe top song in playlist 2 is {top_2["Top Song"].values[0]}')
print(f'The top song from both playlists is {top_both["Top Song"].values[0]}')

The top song in playlist 1 is Flare Guns
The top song in playlist 2 is Location (feat. Burna Boy)
The top song from both playlists is Grey Luh


## Artists

### Which artist has the most songs in each playlist?

One point of contingency is that each value count only will return the unique values. So if an artist has a song which they perform on their own and has another song which
they perform with a group of others, we don't take into account those songs as a count towards the individual artist. We could make the argument that leaving the column in its original format would make it easier to count the values of each artist in the column. 

In [14]:
# Grab first value from each playlist artists column which has the highest count
top_art_1 = playlist_1['artists'].value_counts().head(1).rename_axis('Top Artists').reset_index(name = 'counts')
top_art_2 = playlist_2['artists'].value_counts().head(1).rename_axis('Top Artists').reset_index(name = 'counts')
top_art_both = merge['artists'].value_counts().head(1).rename_axis('Top Artists').reset_index(name = 'counts')

print(f'The top artist(s) in playlist 1 is {top_art_1["Top Artists"].values[0]}\nThe top artist(s) in playlist 2 is {top_art_2["Top Artists"].values[0]}')
print(f'The top artist(s) from both playlists is {top_art_both["Top Artists"].values[0]}')

The top artist(s) in playlist 1 is Quinn XCII
The top artist(s) in playlist 2 is Trettmann / KitschKrieg / SFR
The top artist(s) from both playlists is Quinn XCII


### Visualize the number of songs per artist in each playlist

In [15]:
fig = px.histogram(playlist_1, x = 'artists', title='Number of songs per artist(s) in Playlist 1')
fig.update_layout(title_x = 0.5)

In [16]:
fig = px.histogram(playlist_2, x = 'artists', title= 'Number of songs per artist(s) in Playlist 2')
fig.update_layout(title_x = 0.5)

In [17]:
fig = px.histogram(merge, y='artists', orientation='h', height=4000, title= 'Number of tracks each artist has in both playlists')
fig.update_layout(title_x = 0.5)
fig.show()

In [18]:
"""
By taking only the columns we need and counting the values of each unique occurance of both columns, we can find the number of albums each artist appears in.
Then, taking the value that occurs the most, we can separate the artist(s) that appear in the most albums per playlist.
"""
top_album_art_1 = playlist_1[['artists', 'album']].value_counts().rename_axis(['artists', 'album']).reset_index(name = 'counts')
top_album_art_1 = top_album_art_1['artists'].value_counts().head(1).rename_axis('artists').reset_index(name = 'counts')

top_album_art_2 = playlist_2[['artists', 'album']].value_counts().rename_axis(['artists', 'album']).reset_index(name = 'counts')
top_album_art_2 = top_album_art_2['artists'].value_counts().head(1).rename_axis('artists').reset_index(name = 'counts')

top_album_art_both = merge[['artists', 'album']].value_counts().rename_axis(['artists', 'album']).reset_index(name = 'counts')
top_album_art_both = top_album_art_both['artists'].value_counts().head(1).rename_axis('artists').reset_index(name = 'counts')

print(f'The artist with the most songs from different albums in playlist 1 is {top_album_art_1["artists"].values[0]} with a total of {top_album_art_1["counts"].values[0]} different album appearances')
print(f'The artist with the most songs from different albums in playlist 2 is {top_album_art_2["artists"].values[0]} with a total of {top_album_art_2["counts"].values[0]} different album appearances')
print(f'The artist with the most songs from different albums in both playlists is {top_album_art_both["artists"].values[0]} with a total of {top_album_art_both["counts"].values[0]} different album appearances')

The artist with the most songs from different albums in playlist 1 is Quinn XCII with a total of 6 different album appearances
The artist with the most songs from different albums in playlist 2 is RIZ LA VIE with a total of 3 different album appearances
The artist with the most songs from different albums in both playlists is Quinn XCII with a total of 6 different album appearances


## Explicit or Not?

### Create a visualization depicting which playlist contains a greater percentage of explicit music.

In [19]:
# Create a single DataFrame object containing explicit values for both playlists
explicit_df = pd.DataFrame(data = {'p_1_explicit': playlist_1['explicit'], 'p_2_explicit': playlist_2['explicit']})
# We can fill NaN values in explicit column with False as we are searching for True. NaN values exist because those songs are not in playlist 2
explicit_df.fillna(False, inplace=True)

# Create visualization
px.histogram(explicit_df, x = ['p_1_explicit', 'p_2_explicit'], title = 'Ratio of Explicit Values Across Both Playlists', barmode='group').update_layout(title_x = 0.5, xaxis_title = 'Explicit')

By reviewing the visualization, we can easily deduce that playlist 2 contains much more explicit songs than playlist 1

## Preview URL

### Create a visualization for the preview_url column

In [20]:
px.histogram(merge[['artists','preview_url']].value_counts().rename_axis(['artists', 'preview_url']).reset_index(name = 'counts'), 
             x = 'artists', y = 'counts', title = "Sum of URL's per Artist").update_layout(xaxis_title = 'Artists', 
                                                                                           yaxis_title = "Sum of URL's", title_x = 0.5)